# 虚假新闻二分类（文本基线 + 图像融合 + 预训练微调）

包含：
- 文本基线 TF-IDF+LR（含进度打印与F1阈值搜索）
- 图像HSV特征与文本稀疏拼接的 late fusion（含进度打印）
- 预训练文本微调（RoBERTa/MacBERT 单折示例，含进度打印）
- 视觉主干（ResNet50）与 OpenCLIP 特征提取融合（CPU 环境示例，含进度打印）

In [2]:
# ============================ PART 0: 环境准备 ============================
import os, random, time, warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['http_proxy'] = ''
os.environ['https_proxy'] = ''
os.environ['HTTP_PROXY'] = ''
os.environ['HTTPS_PROXY'] = ''

CACHE = ".cache"
os.makedirs(CACHE, exist_ok=True)


In [3]:
# ============================ PART 1: 数据加载 + 文本预处理 ============================
print("="*60)
print("📊 数据加载中...")
print("="*60)

train_df = pd.read_pickle("train.pkl")
test_df  = pd.read_pickle("test_no_label.pkl")

print(f"✅ 训练集: {train_df.shape[0]} 条样本, {train_df.shape[1]} 个特征")
print(f"✅ 测试集: {test_df.shape[0]} 条样本, {test_df.shape[1]} 个特征")
print(f"📋 训练集列名: {list(train_df.columns)}")

# 标签分布统计
label_counts = train_df['label'].value_counts().sort_index()
print(f"📈 标签分布:")
for label, count in label_counts.items():
    pct = count / len(train_df) * 100
    print(f"   标签 {label}: {count} 条 ({pct:.2f}%)")

text_col = "news body text" if "news body text" in train_df.columns else "text"
print(f"📝 使用文本列: '{text_col}'")

def normalize_text(s, max_len=6000):
    if not isinstance(s, str): return ""
    s = s.replace("\r", " ").replace("\n", " ").strip()
    if len(s) > max_len:
        return s[:max_len//2] + " " + s[-max_len//2:]
    return s

print("\n🔄 文本预处理中...")
train_df["text"] = [normalize_text(t) for t in tqdm(train_df[text_col], desc="训练集文本预处理", total=len(train_df))]
test_df["text"]  = [normalize_text(t) for t in tqdm(test_df[text_col], desc="测试集文本预处理", total=len(test_df))]

# 文本长度统计
train_lens = [len(t) for t in train_df["text"]]
test_lens = [len(t) for t in test_df["text"]]
print(f"\n📏 文本长度统计:")
print(f"   训练集 - 平均: {np.mean(train_lens):.1f}, 中位数: {np.median(train_lens):.1f}, 最大: {np.max(train_lens)}, 最小: {np.min(train_lens)}")
print(f"   测试集 - 平均: {np.mean(test_lens):.1f}, 中位数: {np.median(test_lens):.1f}, 最大: {np.max(test_lens)}, 最小: {np.min(test_lens)}")

X = train_df["text"].values
y = train_df["label"].astype(int).values
X_test = test_df["text"].values

print("\n✅ 文本预处理完成！")


📊 数据加载中...
✅ 训练集: 7000 条样本, 13 个特征
✅ 测试集: 3000 条样本, 11 个特征
📋 训练集列名: ['text', 'image_path', 'entity_id', 'topic', 'fine-grained label', 'label', 'knowledge_embedding', 'description', 'relation', 'platform', 'author', 'date', 'comment']
📈 标签分布:
   标签 0: 4136 条 (59.09%)
   标签 1: 2864 条 (40.91%)
📝 使用文本列: 'text'

🔄 文本预处理中...


测试集文本预处理: 100%|██████████| 3000/3000 [00:00<00:00, 453748.95it/s]


📏 文本长度统计:
   训练集 - 平均: 924.8, 中位数: 121.0, 最大: 6001, 最小: 3
   测试集 - 平均: 975.0, 中位数: 122.0, 最大: 6001, 最小: 6

✅ 文本预处理完成！


## TF-IDF + LogisticRegression 5折基线

In [4]:
# ============================ PART 2: 5折 TF-IDF + LR ============================
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, f1_score, accuracy_score, precision_score, recall_score

print("="*60)
print("🚀 开始 TF-IDF + LogisticRegression 5折交叉验证")
print("="*60)

def build_tfidf_lr():
    vec = TfidfVectorizer(analyzer='char', ngram_range=(2,5),
                          max_features=200000, lowercase=False)
    clf = LogisticRegression(solver='saga', class_weight='balanced',
                             max_iter=350, n_jobs=-1)
    return Pipeline([("tfidf", vec), ("lr", clf)])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

oof_tfidf = np.zeros(len(X))
pred_tfidf = np.zeros(len(X_test))

fold_metrics = []

for fold, (tr, va) in enumerate(tqdm(skf.split(X, y), total=5, desc="5折交叉验证"), 1):
    print(f"\n{'='*60}")
    print(f"📦 Fold {fold}/5 开始训练")
    print(f"{'='*60}")
    print(f"   训练集大小: {len(tr)} ({len(tr)/len(X)*100:.1f}%)")
    print(f"   验证集大小: {len(va)} ({len(va)/len(X)*100:.1f}%)")
    print(f"   训练集标签分布: {np.bincount(y[tr])}")
    
    start_time = time.time()
    model = build_tfidf_lr()
    
    print("   🔄 TF-IDF特征提取和模型训练中...")
    model.fit(X[tr], y[tr])
    
    # 获取特征维度
    n_features = model.named_steps['tfidf'].transform(X[tr][:1]).shape[1]
    print(f"   ✅ 特征维度: {n_features}")
    
    # 验证集预测
    print("   🔄 验证集预测中...")
    oof_tfidf[va] = model.predict_proba(X[va])[:,1]
    
    # 测试集预测
    print("   🔄 测试集预测中...")
    pred_tfidf += model.predict_proba(X_test)[:,1] / 5
    
    # 计算验证集指标
    va_pred_binary = (oof_tfidf[va] >= 0.5).astype(int)
    va_f1 = f1_score(y[va], va_pred_binary)
    va_acc = accuracy_score(y[va], va_pred_binary)
    va_prec = precision_score(y[va], va_pred_binary, zero_division=0)
    va_rec = recall_score(y[va], va_pred_binary, zero_division=0)
    
    fold_metrics.append({
        'fold': fold,
        'f1': va_f1,
        'acc': va_acc,
        'prec': va_prec,
        'rec': va_rec
    })
    
    elapsed = time.time() - start_time
    print(f"   ⏱️  耗时: {elapsed:.2f}秒")
    print(f"   📊 Fold {fold} 验证集指标:")
    print(f"      F1: {va_f1:.5f} | 准确率: {va_acc:.5f} | 精确率: {va_prec:.5f} | 召回率: {va_rec:.5f}")

print(f"\n{'='*60}")
print("📈 5折交叉验证完成，计算最佳阈值...")
print(f"{'='*60}")

p, r, t = precision_recall_curve(y, oof_tfidf)
f1 = 2*p*r/(p+r+1e-8)
thr_tfidf = t[np.argmax(f1)]
best_f1 = f1.max()

print(f"\n🎯 最佳阈值: {thr_tfidf:.4f}")
print(f"🎯 OOF F1 Score: {best_f1:.5f}")

# 使用最佳阈值的指标
best_pred = (oof_tfidf >= thr_tfidf).astype(int)
best_acc = accuracy_score(y, best_pred)
best_prec = precision_score(y, best_pred, zero_division=0)
best_rec = recall_score(y, best_pred, zero_division=0)

print(f"\n📊 使用最佳阈值后的整体指标:")
print(f"   F1: {best_f1:.5f} | 准确率: {best_acc:.5f} | 精确率: {best_prec:.5f} | 召回率: {best_rec:.5f}")

print(f"\n📊 各折详细指标:")
for m in fold_metrics:
    print(f"   Fold {m['fold']}: F1={m['f1']:.5f}, Acc={m['acc']:.5f}, Prec={m['prec']:.5f}, Rec={m['rec']:.5f}")

print(f"\n✅ TF-IDF + LR 训练完成！")


🚀 开始 TF-IDF + LogisticRegression 5折交叉验证


5折交叉验证:   0%|          | 0/5 [00:00<?, ?it/s]


📦 Fold 1/5 开始训练
   训练集大小: 5600 (80.0%)
   验证集大小: 1400 (20.0%)
   训练集标签分布: [3309 2291]
   🔄 TF-IDF特征提取和模型训练中...
   ✅ 特征维度: 200000
   🔄 验证集预测中...
   🔄 测试集预测中...


5折交叉验证:  20%|██        | 1/5 [00:22<01:29, 22.29s/it]

   ⏱️  耗时: 22.27秒
   📊 Fold 1 验证集指标:
      F1: 0.68798 | 准确率: 0.77000 | 精确率: 0.77342 | 召回率: 0.61955

📦 Fold 2/5 开始训练
   训练集大小: 5600 (80.0%)
   验证集大小: 1400 (20.0%)
   训练集标签分布: [3309 2291]
   🔄 TF-IDF特征提取和模型训练中...
   ✅ 特征维度: 200000
   🔄 验证集预测中...
   🔄 测试集预测中...


...

   ⏱️  耗时: 23.26秒
   📊 Fold 3 验证集指标:
      F1: 0.69549 | 准确率: 0.76857 | 精确率: 0.75356 | 召回率: 0.64572

📦 Fold 4/5 开始训练
   训练集大小: 5600 (80.0%)
   验证集大小: 1400 (20.0%)
   训练集标签分布: [3309 2291]
   🔄 TF-IDF特征提取和模型训练中...
   ✅ 特征维度: 200000
   🔄 验证集预测中...
   🔄 测试集预测中...


5折交叉验证:  80%|████████  | 4/5 [01:28<00:22, 22.55s/it]

   ⏱️  耗时: 23.57秒
   📊 Fold 4 验证集指标:
      F1: 0.73529 | 准确率: 0.79429 | 精确率: 0.77670 | 召回率: 0.69808

📦 Fold 5/5 开始训练
   训练集大小: 5600 (80.0%)
   验证集大小: 1400 (20.0%)
   训练集标签分布: [3308 2292]
   🔄 TF-IDF特征提取和模型训练中...
   ✅ 特征维度: 200000
   🔄 验证集预测中...
   🔄 测试集预测中...


5折交叉验证: 100%|██████████| 5/5 [01:52<00:00, 22.50s/it]

   ⏱️  耗时: 23.76秒
   📊 Fold 5 验证集指标:
      F1: 0.70611 | 准确率: 0.78000 | 精确率: 0.77731 | 召回率: 0.64685

📈 5折交叉验证完成，计算最佳阈值...

🎯 最佳阈值: 0.4202
🎯 OOF F1 Score: 0.72810

📊 使用最佳阈值后的整体指标:
   F1: 0.72810 | 准确率: 0.76100 | 精确率: 0.68106 | 召回率: 0.78212

📊 各折详细指标:
   Fold 1: F1=0.68798, Acc=0.77000, Prec=0.77342, Rec=0.61955
   Fold 2: F1=0.73066, Acc=0.79357, Prec=0.78400, Rec=0.68412
   Fold 3: F1=0.69549, Acc=0.76857, Prec=0.75356, Rec=0.64572
   Fold 4: F1=0.73529, Acc=0.79429, Prec=0.77670, Rec=0.69808
   Fold 5: F1=0.70611, Acc=0.78000, Prec=0.77731, Rec=0.64685

✅ TF-IDF + LR 训练完成！


## 图像特征提取（自动缓存 + GPU加速 + 预训练权重）

In [5]:
# ============================ PART 3: 图像特征提取 ============================
import cv2, torch
from PIL import Image
import open_clip
import timm

print("="*60)
print("🖼️  开始图像特征提取")
print("="*60)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  使用设备: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

def safe_load_img(path):
    try:
        img = Image.open(path).convert("RGB")
        return img
    except:
        return None

def extract_clip(paths, save_path, dataset_name=""):
    if os.path.exists(save_path):
        print(f"   ✅ 加载缓存: {save_path}")
        feats = np.load(save_path)['feats']
        print(f"   📊 {dataset_name} CLIP特征形状: {feats.shape}")
        return feats
    
    print(f"   🔄 {dataset_name} 提取 CLIP ViT-B/16 特征...")
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='laion400m_e32')
    model.to(device).eval()
    
    print(f"   📦 模型参数量: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")
    
    feats = []
    failed_count = 0
    start_time = time.time()
    
    with torch.no_grad():
        for p in tqdm(paths, desc=f"   CLIP {dataset_name}", leave=False):
            img = safe_load_img(os.path.join("image", p))
            if img is None:
                feats.append(np.zeros(768, dtype=np.float32))
                failed_count += 1
                continue
            x = preprocess(img).unsqueeze(0).to(device)
            f = model.encode_image(x).float().cpu().numpy()[0]
            feats.append(f)
    
    feats = np.vstack(feats)
    elapsed = time.time() - start_time
    
    print(f"   ✅ {dataset_name} CLIP特征提取完成:")
    print(f"      特征形状: {feats.shape}")
    print(f"      失败图像数: {failed_count} ({failed_count/len(paths)*100:.2f}%)")
    print(f"      耗时: {elapsed:.2f}秒 ({elapsed/len(paths)*1000:.2f}ms/样本)")
    print(f"      保存到: {save_path}")
    
    np.savez_compressed(save_path, feats=feats)
    return feats

def extract_deit(paths, save_path, dataset_name=""):
    if os.path.exists(save_path):
        print(f"   ✅ 加载缓存: {save_path}")
        feats = np.load(save_path)['feats']
        print(f"   📊 {dataset_name} DeiT特征形状: {feats.shape}")
        return feats
    
    print(f"   🔄 {dataset_name} 提取 DeiT-base 特征...")
    model = timm.create_model("deit_base_patch16_224.fb_in1k",
                               pretrained=True, num_classes=0).to(device).eval()
    trans = timm.data.create_transform(**timm.data.resolve_model_data_config(model))
    
    print(f"   📦 模型参数量: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")
    
    feats = []
    failed_count = 0
    start_time = time.time()
    
    with torch.no_grad():
        for p in tqdm(paths, desc=f"   DeiT {dataset_name}", leave=False):
            img = safe_load_img(os.path.join("image", p))
            if img is None:
                feats.append(np.zeros(768, dtype=np.float32))
                failed_count += 1
                continue
            x = trans(img).unsqueeze(0).to(device)
            f = model(x).cpu().numpy()[0]
            feats.append(f)
    
    feats = np.vstack(feats)
    elapsed = time.time() - start_time
    
    print(f"   ✅ {dataset_name} DeiT特征提取完成:")
    print(f"      特征形状: {feats.shape}")
    print(f"      失败图像数: {failed_count} ({failed_count/len(paths)*100:.2f}%)")
    print(f"      耗时: {elapsed:.2f}秒 ({elapsed/len(paths)*1000:.2f}ms/样本)")
    print(f"      保存到: {save_path}")
    
    np.savez_compressed(save_path, feats=feats)
    return feats

print("\n📥 提取训练集图像特征...")
clip_train = extract_clip(train_df['image_path'], f"{CACHE}/clip_train.npz", "训练集")
deit_train = extract_deit(train_df['image_path'], f"{CACHE}/deit_train.npz", "训练集")

print("\n📥 提取测试集图像特征...")
clip_test  = extract_clip(test_df['image_path'],  f"{CACHE}/clip_test.npz", "测试集")
deit_test  = extract_deit(test_df['image_path'],  f"{CACHE}/deit_test.npz", "测试集")

print("\n🔗 融合图像特征...")
img_train = np.hstack([clip_train, deit_train])
img_test  = np.hstack([clip_test,  deit_test])

print(f"\n📊 最终图像特征统计:")
print(f"   训练集: {img_train.shape} (CLIP: {clip_train.shape[1]}维 + DeiT: {deit_train.shape[1]}维)")
print(f"   测试集: {img_test.shape} (CLIP: {clip_test.shape[1]}维 + DeiT: {deit_test.shape[1]}维)")
print(f"   特征均值: 训练集={img_train.mean():.4f}, 测试集={img_test.mean():.4f}")
print(f"   特征标准差: 训练集={img_train.std():.4f}, 测试集={img_test.std():.4f}")

print("\n✅ 图像特征提取完成！")


🖼️  开始图像特征提取
🖥️  使用设备: cuda
   GPU: NVIDIA GeForce RTX 4060 Laptop GPU
   显存: 8.00 GB

📥 提取训练集图像特征...
   ✅ 加载缓存: .cache/clip_train.npz
   📊 训练集 CLIP特征形状: (7000, 512)
   ✅ 加载缓存: .cache/deit_train.npz
   📊 训练集 DeiT特征形状: (7000, 768)

📥 提取测试集图像特征...
   ✅ 加载缓存: .cache/clip_test.npz
   📊 测试集 CLIP特征形状: (3000, 512)
   ✅ 加载缓存: .cache/deit_test.npz
   📊 测试集 DeiT特征形状: (3000, 768)

🔗 融合图像特征...

📊 最终图像特征统计:
   训练集: (7000, 1280) (CLIP: 512维 + DeiT: 768维)
   测试集: (3000, 1280) (CLIP: 512维 + DeiT: 768维)
   特征均值: 训练集=-0.0083, 测试集=-0.0086
   特征标准差: 训练集=0.6828, 测试集=0.6842

✅ 图像特征提取完成！


## 终极晚融合（TFIDF + CLIP + ViT + RoBERTa-large）

In [6]:
# ============================ PART 4: 晚融合 ============================
from scipy import sparse

print("="*60)
print("🔗 开始晚融合模型训练 (TF-IDF + 图像特征)")
print("="*60)

print("\n🔄 TF-IDF特征提取中...")
vec = TfidfVectorizer(analyzer='char', ngram_range=(2,5), max_features=200000)
text_train = vec.fit_transform(X)
text_test  = vec.transform(X_test)

print(f"   ✅ 文本特征形状:")
print(f"      训练集: {text_train.shape}")
print(f"      测试集: {text_test.shape}")
print(f"      特征维度: {text_train.shape[1]}")

print("\n🔗 融合文本和图像特征...")
Xfull = sparse.hstack([text_train, sparse.csr_matrix(img_train)])
Xtest = sparse.hstack([text_test,  sparse.csr_matrix(img_test)])

print(f"   ✅ 融合后特征形状:")
print(f"      训练集: {Xfull.shape} (文本: {text_train.shape[1]} + 图像: {img_train.shape[1]} = {Xfull.shape[1]})")
print(f"      测试集: {Xtest.shape}")
print(f"      稀疏矩阵格式: {Xfull.format}, 非零元素比例: {Xfull.nnz / (Xfull.shape[0] * Xfull.shape[1]) * 100:.4f}%")

print("\n🚀 训练融合模型...")
start_time = time.time()
clf_fusion = LogisticRegression(solver='saga', class_weight='balanced',
                                max_iter=500, n_jobs=-1)
clf_fusion.fit(Xfull, y)
elapsed = time.time() - start_time

print(f"   ✅ 模型训练完成，耗时: {elapsed:.2f}秒")
print(f"   📊 模型系数统计:")
coef = clf_fusion.coef_[0]
print(f"      系数均值: {coef.mean():.4f}")
print(f"      系数标准差: {coef.std():.4f}")
print(f"      非零系数: {(coef != 0).sum()} / {len(coef)}")

print("\n🔄 生成预测概率...")
prob_fusion = clf_fusion.predict_proba(Xtest)[:,1]
train_prob_fusion = clf_fusion.predict_proba(Xfull)[:,1]

# 计算训练集指标
p, r, t = precision_recall_curve(y, train_prob_fusion)
f1 = 2*p*r/(p+r+1e-8)
thr_fusion = t[np.argmax(f1)]
best_f1_fusion = f1.max()

print(f"\n📊 融合模型性能:")
print(f"   最佳阈值: {thr_fusion:.4f}")
print(f"   训练集 F1: {best_f1_fusion:.5f}")
print(f"   预测概率统计:")
print(f"      均值: {prob_fusion.mean():.4f}")
print(f"      标准差: {prob_fusion.std():.4f}")
print(f"      最小值: {prob_fusion.min():.4f}")
print(f"      最大值: {prob_fusion.max():.4f}")

print("\n✅ 晚融合模型训练完成！")


🔗 开始晚融合模型训练 (TF-IDF + 图像特征)

🔄 TF-IDF特征提取中...
   ✅ 文本特征形状:
      训练集: (7000, 200000)
      测试集: (3000, 200000)
      特征维度: 200000

🔗 融合文本和图像特征...
   ✅ 融合后特征形状:
      训练集: (7000, 201280) (文本: 200000 + 图像: 1280 = 201280)
      测试集: (3000, 201280)
      稀疏矩阵格式: csr, 非零元素比例: 1.4459%

🚀 训练融合模型...
   ✅ 模型训练完成，耗时: 134.31秒
   📊 模型系数统计:
      系数均值: 0.0034
      系数标准差: 0.0473
      非零系数: 201280 / 201280

🔄 生成预测概率...

📊 融合模型性能:
   最佳阈值: 0.4647
   训练集 F1: 0.86329
   预测概率统计:
      均值: 0.4479
      标准差: 0.3606
      最小值: 0.0000
      最大值: 0.9997

✅ 晚融合模型训练完成！


## RoBERTa-large 5折 + 自动缓存 + 最终加权融合

In [ ]:
# ============================ PART 5: RoBERTa（Windows优化速度版本） ============================
import torch
from torch.utils.data import Dataset, DataLoader
from torch import amp
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, precision_recall_curve

print("="*60)
print("🤖 开始 RoBERTa 5折交叉验证")
print("="*60)

# 检测操作系统
is_windows = os.name == 'nt'
print(f"🖥️  操作系统: {'Windows' if is_windows else 'Linux/WSL2'}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu_memory = 0
if torch.cuda.is_available():
    print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"   显存: {gpu_memory:.2f} GB")
    torch.backends.cudnn.benchmark = True
    # 启用TF32加速（Ampere架构及以上）
    if torch.cuda.get_device_capability()[0] >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("   ✅ 已启用TF32加速")
    
    # Windows特定优化：设置CUDA设备属性
    if is_windows:
        # 禁用CUDA图优化（Windows上可能有问题）
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
        print("   ✅ 已应用Windows CUDA优化")
else:
    print("   ⚠️  未检测到CUDA，将使用CPU（速度会很慢）")

# ========== 模型选择：Windows上优先使用base版本 ==========
# 选项1: base版本（125M参数，速度快2-3倍，性能略降）
# MODEL = "hfl/chinese-roberta-wwm-ext"  # base版本，更快
# 选项2: large版本（330M参数，速度慢但性能更好）
MODEL = "hfl/chinese-roberta-wwm-ext-large"  # large版本

# Windows上自动使用base版本以提升速度（除非显存充足且用户明确需要large）
if is_windows:
    if torch.cuda.is_available() and gpu_memory >= 10:
        print(f"   💡 Windows环境，显存充足，使用large版本")
    else:
        MODEL = "hfl/chinese-roberta-wwm-ext"
        print(f"   ⚡ Windows环境，自动切换到base版本以提升速度（如需large版本可手动修改）")
elif torch.cuda.is_available() and gpu_memory < 8:
    MODEL = "hfl/chinese-roberta-wwm-ext"
    print(f"   ⚠️  显存 < 8GB，自动切换到base版本以提升速度")
elif torch.cuda.is_available():
    print(f"   💡 显存充足，使用large版本（如需更快速度，可手动改为base版本）")

print(f"\n📥 加载模型和分词器: {MODEL}")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
print(f"   ✅ 词汇表大小: {len(tokenizer)}")

def tokenize_all(texts):
    return tokenizer(list(texts), truncation=True, max_length=512,
                     padding="max_length", return_tensors="pt")

print("\n🔄 预处理所有文本数据...")
train_enc = tokenize_all(X)
test_enc  = tokenize_all(X_test)
print(f"   ✅ 训练集tokenized: {train_enc['input_ids'].shape}")
print(f"   ✅ 测试集tokenized: {test_enc['input_ids'].shape}")

class MySet(Dataset):
    def __init__(self, enc, y=None):
        self.enc = enc
        self.y = y
    def __len__(self): return len(self.enc['input_ids'])
    def __getitem__(self, i):
        item = {k: v[i] for k,v in self.enc.items()}
        if self.y is not None:
            item['labels'] = torch.tensor(self.y[i],dtype=torch.long)
        return item

skf = StratifiedKFold(5, shuffle=True, random_state=SEED)
pred_roberta = np.zeros(len(X_test))
oof_roberta  = np.zeros(len(X))

fold_summaries = []

for fold, (tr, va) in enumerate(skf.split(X, y), 1):
    print(f"\n{'='*60}")
    print(f"📦 Fold {fold}/5 开始")
    print(f"{'='*60}")
    print(f"   训练集: {len(tr)} 样本 ({len(tr)/len(X)*100:.1f}%)")
    print(f"   验证集: {len(va)} 样本 ({len(va)/len(X)*100:.1f}%)")
    print(f"   训练集标签分布: {np.bincount(y[tr])}")
    
    fold_start_time = time.time()
    
    print(f"\n   📥 加载模型...")
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL, num_labels=2, dtype=torch.bfloat16
    ).to(device)
    
    n_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"   ✅ 模型参数量: {n_params/1e6:.1f}M (可训练: {trainable_params/1e6:.1f}M)")

    train_ds = MySet({k:v[tr] for k,v in train_enc.items()}, y[tr])
    val_ds   = MySet({k:v[va] for k,v in train_enc.items()}, y[va])
    test_ds  = MySet(test_enc)

    # ========== 批次大小优化：Windows上使用更保守的设置 ==========
    if not torch.cuda.is_available():
        # CPU模式：小批次
        train_batch_size = 4
        val_batch_size = 8
        grad_accum_steps = 1
    elif is_windows:
        # Windows环境：使用更保守的批次大小以避免问题
        if "large" in MODEL.lower():
            train_batch_size = 20 if gpu_memory >= 10 else 14  # Windows上稍小
            val_batch_size = 40 if gpu_memory >= 10 else 28
            grad_accum_steps = 2
        else:
            train_batch_size = 28 if gpu_memory >= 8 else 20  # Windows上稍小
            val_batch_size = 56 if gpu_memory >= 8 else 40
            grad_accum_steps = 1
    elif "large" in MODEL.lower():
        # Linux/WSL2: large模型
        train_batch_size = 24 if gpu_memory >= 10 else 16
        val_batch_size = 48 if gpu_memory >= 10 else 32
        grad_accum_steps = 2
    else:
        # Linux/WSL2: base模型
        train_batch_size = 32 if gpu_memory >= 8 else 24
        val_batch_size = 64 if gpu_memory >= 8 else 48
        grad_accum_steps = 1
    
    # Windows上必须使用num_workers=0，否则会有问题
    num_workers = 0 if is_windows else 2
    
    # Windows上简化DataLoader配置
    if is_windows:
        train_loader = DataLoader(
            train_ds, 
            batch_size=train_batch_size, 
            shuffle=True, 
            num_workers=0, 
            pin_memory=torch.cuda.is_available()
        )
        val_loader = DataLoader(
            val_ds, 
            batch_size=val_batch_size, 
            num_workers=0, 
            pin_memory=torch.cuda.is_available()
        )
        test_loader = DataLoader(
            test_ds, 
            batch_size=val_batch_size, 
            num_workers=0, 
            pin_memory=torch.cuda.is_available()
        )
    else:
        # Linux/WSL2: 使用优化的DataLoader
        train_loader = DataLoader(
            train_ds, 
            batch_size=train_batch_size, 
            shuffle=True, 
            num_workers=num_workers, 
            pin_memory=True,
            persistent_workers=True,
            prefetch_factor=2
        )
        val_loader = DataLoader(
            val_ds, 
            batch_size=val_batch_size, 
            num_workers=num_workers, 
            pin_memory=True,
            persistent_workers=True,
            prefetch_factor=2
        )
        test_loader = DataLoader(
            test_ds, 
            batch_size=val_batch_size, 
            num_workers=num_workers, 
            pin_memory=True,
            persistent_workers=True,
            prefetch_factor=2
        )

    # Windows上不使用fused优化器（可能不支持）
    if is_windows:
        opt = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
        print(f"   ⚠️  Windows环境，使用标准AdamW（fused在Windows上可能不稳定）")
    else:
        try:
            opt = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01, fused=True)
            print(f"   ✅ 使用fused AdamW优化器")
        except:
            opt = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
            print(f"   ⚠️  fused AdamW不可用，使用标准AdamW")
    
    steps = len(train_loader) * 2 // grad_accum_steps
    sched = get_linear_schedule_with_warmup(opt, 50, steps)
    
    # Windows上不使用torch.compile（通常很慢或不稳定）
    use_compile = False
    # print(f"   ⚠️  已禁用torch.compile（避免embedding相关报错，速度影响很小）")
    if hasattr(torch, 'compile') and not is_windows:
        try:
            model = torch.compile(model, mode='reduce-overhead')
            use_compile = True
            print(f"   ✅ 已启用torch.compile加速")
        except Exception as e:
            print(f"   ⚠️  torch.compile不可用: {e}")
    elif is_windows:
        print(f"   ⚠️  Windows环境，跳过torch.compile（Windows上通常很慢）")
    
    print(f"   📊 训练配置:")
    print(f"      批次大小: {train_batch_size} (训练), {val_batch_size} (验证/测试)")
    print(f"      梯度累积步数: {grad_accum_steps} (有效批次: {train_batch_size * grad_accum_steps})")
    print(f"      数据加载进程数: {num_workers}")
    print(f"      总步数: {steps}")
    print(f"      学习率: 3e-5")
    print(f"      Warmup步数: 50")

    best_f1 = -1
    best_epoch = -1
    epoch_metrics = []

    for epoch in range(2):
        print(f"\n   🔄 Epoch {epoch+1}/2 训练中...")
        model.train()
        
        epoch_losses = []
        epoch_start = time.time()
        
        progress_bar = tqdm(train_loader, desc=f"      Fold {fold} Epoch {epoch+1}", leave=False)
        opt.zero_grad()  # 初始化梯度
        
        for step, batch in enumerate(progress_bar):
            # Windows上使用non_blocking=False可能更稳定
            non_blocking = not is_windows
            batch = {k:v.to(device, non_blocking=non_blocking) for k,v in batch.items()}
            
            with amp.autocast('cuda', dtype=torch.bfloat16):
                loss = model(**batch).loss / grad_accum_steps  # 梯度累积时缩放损失
            
            loss.backward()
            epoch_losses.append(loss.item() * grad_accum_steps)  # 记录真实损失
            
            # 梯度累积：每grad_accum_steps步更新一次
            if (step + 1) % grad_accum_steps == 0 or (step + 1) == len(train_loader):
                opt.step()
                sched.step()
                opt.zero_grad()
                
                # Windows上定期清理缓存以避免内存碎片
                if is_windows and (step + 1) % (grad_accum_steps * 10) == 0:
                    torch.cuda.empty_cache()
            
            current_lr = sched.get_last_lr()[0]
            
            if torch.cuda.is_available():
                gpu_mem = torch.cuda.memory_allocated() / 1024**3
                progress_bar.set_postfix({
                    'loss': f'{epoch_losses[-1]:.4f}',
                    'lr': f'{current_lr:.2e}',
                    'gpu': f'{gpu_mem:.1f}GB',
                    'step': f'{step+1}/{len(train_loader)}'
                })
            else:
                progress_bar.set_postfix({
                    'loss': f'{epoch_losses[-1]:.4f}', 
                    'lr': f'{current_lr:.2e}',
                    'step': f'{step+1}/{len(train_loader)}'
                })
        
        avg_loss = np.mean(epoch_losses)
        epoch_time = time.time() - epoch_start
        print(f"      ✅ Epoch {epoch+1} 完成: 平均损失={avg_loss:.4f}, 耗时={epoch_time:.1f}秒")

        # --------- 验证 ---------
        print(f"   🔍 Epoch {epoch+1} 验证中...")
        model.eval()
        preds = []
        trues = []
        with torch.no_grad():
            non_blocking = not is_windows
            for batch in tqdm(val_loader, desc="      验证", leave=False):
                b = {k:v.to(device, non_blocking=non_blocking) for k,v in batch.items()}
                with amp.autocast('cuda', dtype=torch.bfloat16):
                    logits = model(**b).logits
                p = torch.softmax(logits, dim=-1)[:,1]
                # bfloat16需要先转换为float32才能转numpy
                preds.append(p.float().cpu().numpy())
                trues.append(b['labels'].cpu().numpy())
        
        preds = np.concatenate(preds)
        trues = np.concatenate(trues)
        
        # 计算多个指标
        pred_binary = (preds >= 0.5).astype(int)
        f1 = f1_score(trues, pred_binary)
        acc = accuracy_score(trues, pred_binary)
        prec = precision_score(trues, pred_binary, zero_division=0)
        rec = recall_score(trues, pred_binary, zero_division=0)
        
        epoch_metrics.append({
            'epoch': epoch+1,
            'loss': avg_loss,
            'f1': f1,
            'acc': acc,
            'prec': prec,
            'rec': rec
        })
        
        print(f"      📊 Epoch {epoch+1} 验证指标:")
        print(f"         F1: {f1:.5f} | 准确率: {acc:.5f} | 精确率: {prec:.5f} | 召回率: {rec:.5f}")
        
        if f1 > best_f1:
            best_f1 = f1
            best_epoch = epoch + 1
            # 如果使用了compile，需要获取原始模型的state_dict
            if use_compile:
                # torch.compile后的模型有_orig_mod属性，保存原始模型的权重
                state_dict_to_save = model._orig_mod.state_dict()
            else:
                state_dict_to_save = model.state_dict()
            torch.save(state_dict_to_save, f"{CACHE}/best_model_fold{fold}.pt")
            print(f"      🔥 保存最佳模型 (F1={f1:.5f})")

    # 加载最佳模型
    print(f"\n   📥 加载最佳模型 (Epoch {best_epoch})...")
    best_model = AutoModelForSequenceClassification.from_pretrained(
        MODEL, num_labels=2, dtype=torch.bfloat16
    ).to(device)
    
    # 加载state_dict并处理可能的_orig_mod前缀
    saved_state_dict = torch.load(f"{CACHE}/best_model_fold{fold}.pt")
    
    # 检查是否有_orig_mod前缀（可能是之前保存的compile模型）
    if any(k.startswith('_orig_mod.') for k in saved_state_dict.keys()):
        # 移除_orig_mod前缀
        new_state_dict = {}
        for k, v in saved_state_dict.items():
            if k.startswith('_orig_mod.'):
                new_state_dict[k.replace('_orig_mod.', '')] = v
            else:
                new_state_dict[k] = v
        saved_state_dict = new_state_dict
        print(f"   ✅ 已处理state_dict中的_orig_mod前缀")
    
    best_model.load_state_dict(saved_state_dict, strict=False)
    
    # 如果使用了compile，也需要compile最佳模型
    if use_compile:
        best_model = torch.compile(best_model, mode='reduce-overhead')

    # 验证集概率
    print(f"   🔄 生成验证集预测...")
    best_model.eval()
    preds = []
    with torch.no_grad():
        non_blocking = not is_windows
        for batch in tqdm(val_loader, desc="      验证集预测", leave=False):
            b = {k:v.to(device, non_blocking=non_blocking) for k,v in batch.items()}
            with amp.autocast('cuda', dtype=torch.bfloat16):
                logits = best_model(**b).logits
            # bfloat16需要先转换为float32才能转numpy
            preds.append(torch.softmax(logits, dim=-1)[:,1].float().cpu().numpy())
    oof_roberta[va] = np.concatenate(preds)
    
    # 计算验证集最佳阈值
    p, r, t = precision_recall_curve(y[va], oof_roberta[va])
    f1_curve = 2*p*r/(p+r+1e-12)
    best_thr = t[np.argmax(f1_curve)]
    best_f1_thr = f1_curve.max()
    print(f"   🎯 验证集最佳阈值: {best_thr:.4f}, F1: {best_f1_thr:.5f}")

    # test 预测
    print(f"   🔄 生成测试集预测...")
    preds = []
    with torch.no_grad():
        non_blocking = not is_windows
        for batch in tqdm(test_loader, desc="      测试集预测", leave=False):
            b = {k:v.to(device, non_blocking=non_blocking) for k,v in batch.items()}
            with amp.autocast('cuda', dtype=torch.bfloat16):
                logits = best_model(**b).logits
            # bfloat16需要先转换为float32才能转numpy
            preds.append(torch.softmax(logits, dim=-1)[:,1].float().cpu().numpy())
    pred_roberta += np.concatenate(preds) / 5
    
    fold_time = time.time() - fold_start_time
    print(f"\n   ✅ Fold {fold} 完成，总耗时: {fold_time/60:.1f}分钟")
    print(f"   📊 Fold {fold} 最佳指标 (Epoch {best_epoch}):")
    best_epoch_metrics = epoch_metrics[best_epoch-1]
    print(f"      F1: {best_epoch_metrics['f1']:.5f} | 准确率: {best_epoch_metrics['acc']:.5f}")
    print(f"      精确率: {best_epoch_metrics['prec']:.5f} | 召回率: {best_epoch_metrics['rec']:.5f}")
    
    fold_summaries.append({
        'fold': fold,
        'best_epoch': best_epoch,
        'best_f1': best_f1,
        'best_thr': best_thr,
        'time': fold_time
    })
    
    # 清理显存
    del model, best_model, opt, sched, train_loader, val_loader, test_loader
    torch.cuda.empty_cache()

print(f"\n{'='*60}")
print("📈 RoBERTa 5折交叉验证完成")
print(f"{'='*60}")
print(f"\n📊 各折总结:")
for s in fold_summaries:
    print(f"   Fold {s['fold']}: 最佳Epoch={s['best_epoch']}, F1={s['best_f1']:.5f}, "
          f"最佳阈值={s['best_thr']:.4f}, 耗时={s['time']/60:.1f}分钟")

# 计算整体OOF指标
p, r, t = precision_recall_curve(y, oof_roberta)
f1_curve = 2*p*r/(p+r+1e-12)
best_thr_roberta = t[np.argmax(f1_curve)]
best_f1_roberta = f1_curve.max()

print(f"\n🎯 RoBERTa整体OOF指标:")
print(f"   最佳阈值: {best_thr_roberta:.4f}")
print(f"   OOF F1: {best_f1_roberta:.5f}")
print(f"   预测概率统计:")
print(f"      均值: {pred_roberta.mean():.4f}")
print(f"      标准差: {pred_roberta.std():.4f}")

print("\n✅ RoBERTa训练完成！")


🤖 开始 RoBERTa 5折交叉验证
🖥️  操作系统: Linux/WSL2
🖥️  GPU: NVIDIA GeForce RTX 4060 Laptop GPU
   显存: 8.00 GB
   ✅ 已启用TF32加速
   💡 显存充足，使用large版本（如需更快速度，可手动改为base版本）

📥 加载模型和分词器: hfl/chinese-roberta-wwm-ext-large
   ✅ 词汇表大小: 21128

🔄 预处理所有文本数据...
   ✅ 训练集tokenized: torch.Size([7000, 512])
   ✅ 测试集tokenized: torch.Size([3000, 512])

📦 Fold 1/5 开始
   训练集: 5600 样本 (80.0%)
   验证集: 1400 样本 (20.0%)
   训练集标签分布: [3309 2291]

   📥 加载模型...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   ✅ 模型参数量: 325.5M (可训练: 325.5M)
   ✅ 使用fused AdamW优化器
   ⚠️  已禁用torch.compile（避免embedding相关报错，速度影响很小）
   📊 训练配置:
      批次大小: 16 (训练), 32 (验证/测试)
      梯度累积步数: 2 (有效批次: 32)
      数据加载进程数: 2
      总步数: 350
      学习率: 3e-5
      Warmup步数: 50

   🔄 Epoch 1/2 训练中...


      Fold 1 Epoch 1:   3%|▎         | 10/350 [09:00<5:27:39, 57.82s/it, loss=0.6359, lr=3.00e-06, gpu=3.0GB, step=10/350]

In [ ]:
# ============================ PART 6: 最终融合 ============================
print("="*60)
print("🔗 最终模型融合")
print("="*60)

print("\n📊 融合权重:")
print("   晚融合模型 (TF-IDF + 图像): 34%")
print("   RoBERTa模型: 66%")

final_prob = 0.34*prob_fusion + 0.66*pred_roberta

print(f"\n📈 融合后预测概率统计:")
print(f"   均值: {final_prob.mean():.4f}")
print(f"   标准差: {final_prob.std():.4f}")
print(f"   最小值: {final_prob.min():.4f}")
print(f"   最大值: {final_prob.max():.4f}")
print(f"   中位数: {np.median(final_prob):.4f}")

# 使用训练集融合概率找最佳阈值
train_final_prob = 0.34*train_prob_fusion + 0.66*oof_roberta
p, r, t = precision_recall_curve(y, train_final_prob)
f1_curve = 2*p*r/(p+r+1e-12)
best_thr_final = t[np.argmax(f1_curve)]
best_f1_final = f1_curve.max()

print(f"\n🎯 最佳阈值分析:")
print(f"   最佳阈值: {best_thr_final:.4f}")
print(f"   训练集F1: {best_f1_final:.5f}")

# 使用0.52阈值（用户指定）
thr_used = 0.52
final_pred = (final_prob >= thr_used).astype(int)

print(f"\n📊 使用阈值 {thr_used} 的预测统计:")
print(f"   预测为正类: {final_pred.sum()} ({final_pred.sum()/len(final_pred)*100:.2f}%)")
print(f"   预测为负类: {(1-final_pred).sum()} ({(1-final_pred).sum()/len(final_pred)*100:.2f}%)")

output_file = "FINAL_SUBMISSION.csv"
pd.DataFrame({
    "id": test_df.index,
    "label": final_pred
}).to_csv(output_file, index=False)

print(f"\n✅ 已保存最终提交文件: {output_file}")
print(f"   样本数: {len(final_pred)}")
print(f"   正类数: {final_pred.sum()}")
print(f"   负类数: {(1-final_pred).sum()}")

print("\n🎉 全部流程完成！")


🔗 最终模型融合

📊 融合权重:
   晚融合模型 (TF-IDF + 图像): 34%
   RoBERTa模型: 66%

📈 融合后预测概率统计:
   均值: 0.4326
   标准差: 0.2734
   最小值: 0.0400
   最大值: 0.9086
   中位数: 0.3627

🎯 最佳阈值分析:
   最佳阈值: 0.3666
   训练集F1: 0.82828

📊 使用阈值 0.52 的预测统计:
   预测为正类: 982 (32.73%)
   预测为负类: 2018 (67.27%)

✅ 已保存最终提交文件: FINAL_SUBMISSION.csv
   样本数: 3000
   正类数: 982
   负类数: 2018

🎉 全部流程完成！
